# REGRESSION MODELS FOR THE PREDICTION OF THE ALCOHOL DIFUSSION MODEL PARAMETERS

# READ DATASET

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [196]:
covariates=pd.read_csv('PersonLevel_Master_2-3-21_ES_Edited_Merged_SummStat.csv',index_col='ID_S')
covariates.to_csv("CovariatesCopy.csv",na_rep='NA')
covariates_copy=pd.read_csv('CovariatesCopy.csv',index_col='ID_S')

# EXPLORE CATEGORICAL COLUMNS

In [ ]:
covariates['BIOSEX'].value_counts()
covariates.BIOSEX[covariates['BIOSEX']=='female']=0
covariates.BIOSEX[covariates['BIOSEX']=='male']=1
covariates['BIOSEX'].value_counts()

In [ ]:
covariates['ATE_C'].value_counts()
covariates.ATE_C[covariates['ATE_C']=='8+ hours']=5
covariates.ATE_C[covariates['ATE_C']=='6-7 hours']=4
covariates.ATE_C[covariates['ATE_C']=='5-6 hours']=3
covariates.ATE_C[covariates['ATE_C']=='4-5 hours']=2
covariates.ATE_C[covariates['ATE_C']=='<4 hours']=1
covariates.ATE_C[covariates['ATE_C']=='-99.00']=5
covariates['ATE_C'].value_counts()

In [ ]:
covariates['WATER_C'].value_counts()
covariates.WATER_C[covariates['WATER_C']=='3+ hours']=4
covariates.WATER_C[covariates['WATER_C']=='1-2 hours']=2
covariates.WATER_C[covariates['WATER_C']=='2-3 hours']=3
covariates.WATER_C[covariates['WATER_C']=='<1 hours']=1
covariates['WATER_C'].value_counts()

# TRANSDERMAL ALCOHOL CONCENTRATION (TAC) RELATED COVARIATES

TAC values

In [200]:
import scipy.io

data = scipy.io.loadmat('TAC_inter.mat')
data_list = [[element for element in upperElement] for upperElement in data['TAC_inter']]
TAC1_l=[]
TAC2_l=[]
TAC3_l=[]
TAC4_l=[]
for row in range(len(data_list)):
    TAC1_l.append(data_list[row][0])
    TAC2_l.append(data_list[row][1])
    TAC3_l.append(data_list[row][2])
    TAC4_l.append(data_list[row][3])
newData = list(zip(TAC1_l, TAC2_l, TAC3_l, TAC4_l))
columns = ['TAC1', 'TAC2','TAC3','TAC4']
TAC_inter = pd.DataFrame(newData, columns=columns)

In [ ]:
covariates.entry[covariates['entry'].notna()].astype(int)
index = TAC_inter.index
number_of_rows = len(index) 
for row in range(number_of_rows):
    covariates.TAC1[covariates.entry==row+1]=TAC_inter.TAC1[row]
    covariates.TAC2[covariates.entry==row+1]=TAC_inter.TAC2[row]
    covariates.TAC3[covariates.entry==row+1]=TAC_inter.TAC3[row]
    covariates.TAC4[covariates.entry==row+1]=TAC_inter.TAC4[row]

TAC curve characteristics

In [ ]:
index = covariates.index
number_of_rows = len(index) 
for row in range(number_of_rows):
    if covariates.arm[row]=='R':
        covariates.Peak_TAC[row]= covariates.Peak_TACRarm[row]
        covariates.TAC_Dur[row]= covariates.TACRarm_Dur[row]
        covariates.TAC_AUC[row]= covariates.TACRarm_AUC[row]
        covariates.TAC_0toPeak[row]= covariates.TACRarm_0toPeak[row]
    elif covariates.arm[row]=='L':
        covariates.Peak_TAC[row]= covariates.Peak_TACLarm[row]
        covariates.TAC_Dur[row]= covariates.TACLarm_Dur[row]
        covariates.TAC_AUC[row]= covariates.TACLarm_AUC[row]
        covariates.TAC_0toPeak[row]= covariates.TACLarm_0toPeak[row]

TAC curve slopes

In [203]:
data = scipy.io.loadmat('TAC_slopes.mat')
slopes_list = [[element for element in upperElement] for upperElement in data['TAC_slopes']]
asc_l=[]
desc_l=[]
for row in range(len(slopes_list)):
    asc_l.append(slopes_list[row][0])
    desc_l.append(slopes_list[row][1])
newData = list(zip(asc_l, desc_l))
columns = ['asc_slope', 'desc_slope']
slopes = pd.DataFrame(newData, columns=columns)

In [ ]:
covariates.entry[covariates['entry'].notna()].astype(int)
index = slopes.index
number_of_rows = len(index) 
for row in range(number_of_rows):
    covariates.ASC[covariates.entry==row+1]=slopes.asc_slope[row]
    covariates.DESC[covariates.entry==row+1]=slopes.desc_slope[row]

# X AND y FOR : MODEL DEVELOPMENT / PREDICTION ON TEST SET / PREDICTION ON NEW INDIVIDUAL 

Model development 

In [205]:
y1=covariates.q1_2_A_D_1.iloc[22::].values
y2=covariates.q2_2_A_D_1.iloc[22::].values

In [206]:
y_biv=list(zip(covariates['q1_2_A_D_1'],covariates['q2_2_A_D_1']))
y_biv=y_biv[22::]

In [207]:
cols=['Sess_Type','BIOSEX','Eth_Id_C','HT','WT','BF','VF','RM','WAIST','HIPS','AGE','mLALC','CAL_LUN','ATE_C','WATER_C','TLFB28d_F','TLFB90d_F','TLFB28d_B','TLFB90d_B','Peak_TAC','TAC_Dur','TAC_AUC','TAC_0toPeak','TAC1','TAC2','TAC3','TAC4']
X=covariates[cols]
X=X.iloc[22::,:]
X.TAC4[X['TAC4'].isnull()]=0
X.TAC3[X['TAC3'].isnull()]=0

Predictions on test set

In [ ]:
test_ind=covariates[(covariates.entry==34) | (covariates.entry==50) | (covariates.entry==82)].index

In [11]:
cols=['entry','Sess_Type','BIOSEX','Eth_Id_C','HT','WT','BF','VF','RM','WAIST','HIPS','AGE','mLALC','CAL_LUN','ATE_C','WATER_C','TLFB28d_F','TLFB90d_F','TLFB28d_B','TLFB90d_B','Peak_TAC','TAC_Dur','TAC_AUC','TAC_0toPeak','TAC1','TAC2','TAC3','TAC4']
X=covariates[cols]
X=X.iloc[22::,:]
X.TAC4[X['TAC4'].isnull()]=0
X.TAC3[X['TAC3'].isnull()]=0

In [ ]:
y1=covariates.q1_2_A_D_1.iloc[22::]
y2=covariates.q2_2_A_D_1.iloc[22::]
y1=y1.drop(test_ind).values
y2=y2.drop(test_ind).values

In [214]:
y_biv=list(zip(y1,y2))

Prediction on new individual (first run model development)

Note : TAC_Dur = last nonzero obs time - first nonzero obs time,
       TAC_0toPeak = peak time - last zero obs time (beginning)

In [179]:
new = {'Sess_Type':'First Single', 'BIOSEX':0, 'Eth_Id_C':'Asian', 'HT':166.5, 'WT':49.7, 'BF':19.8, 'VF':2, 'RM':1223,
       'WAIST':64, 'HIPS':83.5, 'AGE':25, 'mLALC':119.95, 'CAL_LUN':587.13, 'ATE_C':5, 'WATER_C':4,
       'TLFB28d_F':1, 'TLFB90d_F':2, 'TLFB28d_B':0, 'TLFB90d_B':0, 'Peak_TAC':0.019,
       'TAC_Dur':117.2520, 'TAC_AUC':1.6219, 'TAC_0toPeak':106.2840, 'TAC1':0.0044, 'TAC2':0.0151, 'TAC3':0.0083, 'TAC4':0, 'ASC':0.0191, 'DESC':-0.0075}
X = X.append(new, ignore_index = True)

# HISTOGRAMS 

In [ ]:
fig, axs = plt.subplots(4, 4, sharey=True, figsize=(15,15), tight_layout=True)

axs[0,0].hist(X['HT'])
axs[0,0].set_title('Height')
axs[1,0].hist(X['WT'])
axs[1,0].set_title('Weight')
axs[2,0].hist(X['BF'])
axs[2,0].set_title('Body Fat')
axs[3,0].hist(X['TLFB28d_F'])
axs[3,0].set_title('28d drink freq')
axs[0,1].hist(X['VF'])
axs[0,1].set_title('Visceral Fat')
axs[0,2].hist(X['RM'])
axs[0,2].set_title('Resting Metabolism')
axs[0,3].hist(X['TLFB90d_F'])
axs[0,3].set_title('90d drink freq')
axs[1,1].hist(X['WAIST'])
axs[1,1].set_title('Waist')
axs[1,2].hist(X['HIPS'])
axs[1,2].set_title('Hips')
axs[1,3].hist(X['TLFB28d_B'])
axs[1,3].set_title('28d binge days')
axs[2,1].hist(X['CAL_LUN'])
axs[2,1].set_title('Lunch Calories')
axs[2,2].hist(X['ATE_C'])
axs[2,2].set_title('Hours from eating')
axs[2,3].hist(X['TLFB90d_B'])
axs[2,3].set_title('90d binge days')
axs[3,1].hist(X['AGE'])
axs[3,1].set_title('Age')
axs[3,2].hist(X['Sess_Type'])
axs[3,2].set_title('Sess Type')
axs[3,3].hist(X['Eth_Id_C'])
axs[3,3].set_title('Ethnicity')

# FEATURE SCALING 

In [208]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X['HT']= scaler.fit_transform(X['HT'].values.reshape(-1, 1))
X['WT']= scaler.fit_transform(X['WT'].values.reshape(-1, 1))
X['BF']= scaler.fit_transform(X['BF'].values.reshape(-1, 1))
X['VF']= scaler.fit_transform(X['VF'].values.reshape(-1, 1))
X['RM']= scaler.fit_transform(X['RM'].values.reshape(-1, 1))
X['WAIST']= scaler.fit_transform(X['WAIST'].values.reshape(-1, 1))
X['HIPS']= scaler.fit_transform(X['HIPS'].values.reshape(-1, 1))
X['AGE']= scaler.fit_transform(X['AGE'].values.reshape(-1, 1))
X['mLALC']= scaler.fit_transform(X['mLALC'].values.reshape(-1, 1))
X['CAL_LUN']= scaler.fit_transform(X['CAL_LUN'].values.reshape(-1, 1))
X['ATE_C']= scaler.fit_transform(X['ATE_C'].values.reshape(-1, 1))
X['WATER_C']= scaler.fit_transform(X['WATER_C'].values.reshape(-1, 1))
X['TLFB28d_F']= scaler.fit_transform(X['TLFB28d_F'].values.reshape(-1, 1))
X['TLFB28d_B']= scaler.fit_transform(X['TLFB28d_B'].values.reshape(-1, 1))
X['TLFB90d_F']= scaler.fit_transform(X['TLFB90d_F'].values.reshape(-1, 1))
X['TLFB90d_B']= scaler.fit_transform(X['TLFB90d_B'].values.reshape(-1, 1))
X['Peak_TAC']= scaler.fit_transform(X['Peak_TAC'].values.reshape(-1, 1))
X['TAC_Dur']= scaler.fit_transform(X['TAC_Dur'].values.reshape(-1, 1))
X['TAC_AUC']= scaler.fit_transform(X['TAC_AUC'].values.reshape(-1, 1))
X['TAC_0toPeak']= scaler.fit_transform(X['TAC_0toPeak'].values.reshape(-1, 1))
X['TAC1']= scaler.fit_transform(X['TAC1'].values.reshape(-1, 1))
X['TAC2']= scaler.fit_transform(X['TAC2'].values.reshape(-1, 1))
X['TAC3']= scaler.fit_transform(X['TAC3'].values.reshape(-1, 1))
X['TAC4']= scaler.fit_transform(X['TAC4'].values.reshape(-1, 1))

# CATEGORICAL FEATURES - ONE-HOT ENCODING

In [209]:
X=pd.get_dummies(X)

In [210]:
X.drop('BIOSEX_1', axis=1, inplace=True)

# REGRESSION MODELS

# LINEAR MODEL

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import statsmodels.api as sm

PREDICTIONS ON SPECIFIC SESSIONS 

In [39]:
def test_sess_preds(X, y, X_test, x_columns, model):
    X_train=X[x_columns]
    X_test=X_test[x_columns]
    mod = model.fit(X_train, y)
    y_pred = mod.predict(X_test)
    print(y_pred)

In [ ]:
X_test=X.loc[test_ind]
X_test=X_test.drop('entry',axis=1)
X=X.drop(test_ind)
X=X.drop('entry',axis=1)
x_columns=['HT', 'WT', 'BF', 'mLALC', 'ATE_C', 'TLFB90d_B', 'TAC_AUC', 'TAC_0toPeak', 'TAC3', 'Sess_Type_First Single', 'BIOSEX_0', 'Eth_Id_C_African/Af-Amer', 'Eth_Id_C_Caucasian', 'Eth_Id_C_Hispanic']
model=LinearRegression()
y=y2

In [ ]:
test_sess_preds(X, y, X_test, x_columns, model)

PREDICTIONS ON NEW INDIVIDUAL

In [ ]:
q1_columns= ['WT', 'BF', 'VF', 'RM', 'TLFB90d_F', 'Peak_TAC', 'TAC_AUC', 'TAC1', 'TAC4', 'Sess_Type_Second single', 'Eth_Id_C_Hispanic']
q2_columns=['HT', 'WT', 'BF', 'mLALC','ATE_C','TLFB90d_B', 'TAC_AUC','TAC_0toPeak',  'TAC3', 'Sess_Type_First Single', 'BIOSEX_0', 'Eth_Id_C_African/Af-Amer', 'Eth_Id_C_Caucasian', 'Eth_Id_C_Hispanic']
model=LinearRegression()
y=y2
X_train=X[q2_columns].iloc[0:261]
X_test=X[q2_columns].iloc[261].values.reshape(1,-1)

In [ ]:
test_sess_preds(X_train, y, X_test, q2_columns, model)

STEPWISE REGRESSION WITHOUT INTERCEPT 

In [ ]:
def get_stats(X,x_columns,y):
    X_OLS = X[x_columns]
    results = sm.OLS(y, X_OLS).fit()
    print(results.summary())

STEPWISE REGRESSION WITH INTERCEPT 

In [ ]:
def get_stats_inter(X,x_columns,y):
    X_OLS = X[x_columns]
    X_OLS = sm.add_constant(X_OLS)
    results = sm.OLS(y, X_OLS).fit()
    print(results.summary())

FORWARD/BACKWARD FEATURE SELECTION

In [ ]:
!pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [115]:
def feature_selection(X_fb,y,model,feat_num,fb):
    
    sfs1 = sfs(model, k_features=feat_num, forward=fb, verbose=2, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X_fb, y)
    feat_names = list(sfs1.k_feature_names_)
    print(feat_names)
    plot_sfs(sfs1.get_metric_dict(), kind=None, figsize=(8, 8))
    #plot_sfs(sfs1.get_metric_dict(), kind='std_err',figsize=(10, 10))
    plt.ylabel('Negative MSE')
    plt.xlabel('Number of features')
    plt.title('Forward Feature Selection')
    plt.grid()
    plt.tight_layout() 
    plt.show()

In [ ]:
feat_num=11
fb=True
X_fb=X[x_columns]
y=y2
model = LinearRegression()

feature_selection(X_fb,y,model,feat_num,fb)

TRAIN/TEST SET SPLIT AND EVALUATION

In [125]:
import math

In [112]:
def test_tr_split(X_tt,y):
    X_train, X_test, y_train, y_test = train_test_split(X_tt, y, test_size = 0.20, random_state = 1)
    lm = model.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Train Set Mean Squared Error: " ,mean_squared_error(y_train, lm.predict(X_train)))
    print("Test Set Mean Sqaured Error: ", mean_squared_error(y_test, lm.predict(X_test)))
    print("Train Set Mean Absolute Error: " ,mean_absolute_error(y_train, lm.predict(X_train)))
    print("Test Set Mean Absolute Error: ", mean_absolute_error(y_test, lm.predict(X_test)))
    res = "\n".join("{} {}".format(x, y) for x, y in zip(y_test, y_pred))
    print(res)

In [ ]:
model=LinearRegression()
X_tt=X[['BF', 'CAL_LUN', 'TLFB28d_F', 'Peak_TAC', 'TAC_AUC', 'TAC_0toPeak', 'TAC1', 'TAC3', 'TAC4', 'Eth_Id_C_Hispanic']]
y=y1

test_tr_split(X_tt,y)

K-FOLD CV 

In [60]:
def cross_val(X_cv,y,model):
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    scores = cross_val_score(model, X_cv, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    print(scores)
    print(scores.std())
    print('Mean absolute error : ', mean(absolute(scores)))
    scores = cross_val_score(model, X_cv, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
    print('Mean squared error : ', mean(absolute(scores)))

In [ ]:
model=LinearRegression()
X_cv=X[['WT', 'BF', 'VF', 'RM', 'TLFB90d_F', 'Peak_TAC', 'TAC_AUC', 'TAC1', 'TAC4', 'Sess_Type_Second single', 'Eth_Id_C_Hispanic']]
y=y1

cross_val(X_cv,y,model)

Cross validation fo estimating std

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold

In [108]:
# evaluate model by averaging performance across each fold

def cv_for_std(X,y,model):

    stdd = list()
    ave_std = list()
    kfold = KFold(n_splits=5, shuffle=True)

    for counter in range(10):
        for train_ix, test_ix in kfold.split(X):
            train_X, test_X = X.iloc[train_ix], X.iloc[test_ix]
            train_y, test_y = y[train_ix], y[test_ix]
            model.fit(train_X, train_y)
            yhat = model.predict(test_X)
            std= np.sqrt((sum((np.subtract(yhat,test_y))**2))/(len(yhat)-1))
            stdd.append(std)
            print('> ', std)
        ave_std.append(mean(stdd))
        stdd = list()
    print('Average std for each round of CV:', ave_std)
    print('Average std of the rounds of CV:', mean(ave_std))

In [ ]:
y=y1
model=LinearRegression()

cv_for_std(X,y,model)

PEARSON CORRELATION 

In [ ]:
from seaborn import heatmap

corr = X.corr() 

plt.figure(figsize = (30, 30))
heatmap(corr, cmap = 'coolwarm',annot=True)
plt.show()

GOLDFELD-QUANDT HOMOSKEDASTICITY TEST

In [72]:
import statsmodels.stats.api as sms

GQ_test = sms.diagnostic.het_goldfeldquandt(y1,X)
df = pd.Series({'F':GQ_test[0], 'Prob>F':GQ_test[1]})
print(df)

F         1.292044
Prob>F    0.229485
dtype: float64


NORMALITY CHECK

In [78]:
from scipy.stats import norm, skew
import seaborn as sns

In [87]:
def get_hist(X,col):
    sns.distplot(X[col], fit = norm, kde = True, color = 'blue')
    plt.plot()
    print('The skew of this distribution is = ', skew(X[col]))

# RANDOM FOREST

In [184]:
from pprint import pprint
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.inspection import permutation_importance

In [212]:
def random_forest(array,y):
    
    global X_train, X_test, y_train, y_test, y_pred_tr, y_pred_test
    
    X_train, X_test, y_train, y_test = train_test_split(array, y, test_size=0.2, random_state=1)

    rf = RandomForestRegressor(random_state=1)
    pprint(rf.get_params())

    rf.fit(X_train, y_train)

    y_pred_test = rf.predict(X_test)
    y_pred_tr = rf.predict(X_train)
    error_test = mean_absolute_error(y_test, y_pred_test) 
    error_tr = mean_absolute_error(y_train, y_pred_tr) 
    #mape_tr = 100 * (error_tr / y_train)
    #mape_test = 100 * (error_test / y_test)
    #accuracy_tr = 100 - np.mean(mape_tr)
    #accuracy_test = 100 - np.mean(mape_test)

    #print('R^2 :', rf.score(X_test, y_test))
    #print('Accuracy training:', round(accuracy_tr, 2), '%.')
    #print('Accuracy test:', round(accuracy_test, 2), '%.')

    print('Mean Absolute Error Train:', error_tr)  
    print('Mean Squared Error Train:', mean_squared_error(y_train, y_pred_tr))  
    print('Mean Absolute Error Test:', error_test)  
    print('Mean Squared Error Test:', mean_squared_error(y_test, y_pred_test))  
    
    feature_importances = list(zip(array, rf.feature_importances_))
    feature_importances_ranked = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    [print('Feature: {:35} Importance: {}'.format(*pair)) for pair in feature_importances_ranked];
    
    feature_names = [i[0] for i in feature_importances_ranked]
    y_ticks = np.arange(0, len(feature_names))
    x_axis = [i[1] for i in feature_importances_ranked]
    plt.figure(figsize = (10, 10))
    plt.barh(feature_names, x_axis)   
    plt.title('Random Forest Feature Importance - q2',
    fontdict= {'fontname':'Comic Sans MS','fontsize' : 20})
    plt.ylabel('Features',fontdict= {'fontsize' : 16})
    plt.show()
    
    result = permutation_importance(rf, X_train, y_train, n_repeats=10,random_state=1)  

In [62]:
def random_forest_meta_estimator(array,y):
    global X_train, X_test, y_train, y_test, y_pred_tr, y_pred_test
    
    X_train, X_test, y_train, y_test = train_test_split(array, y, test_size=0.2, random_state=1)
    
    rf = MultiOutputRegressor(RandomForestRegressor(random_state=1))
    pprint(rf.get_params())

    rf.fit(X_train, y_train)

    y_pred_test = rf.predict(X_test)
    y_pred_tr = rf.predict(X_train)
    error_test = mean_absolute_error(y_test, y_pred_test) 
    error_tr = mean_absolute_error(y_train, y_pred_tr) 
    
    print('Mean Absolute Error Train:', error_tr)  
    print('Mean Squared Error Train:', mean_squared_error(y_train, y_pred_tr))  
    print('Mean Absolute Error Test:', error_test)  
    print('Mean Squared Error Test:', mean_squared_error(y_test, y_pred_test))  

In [ ]:
array=X[['HT','WT','BF','VF','RM','WAIST','HIPS','AGE','mLALC','CAL_LUN','ATE_C','WATER_C','TLFB28d_F','TLFB90d_F','TLFB28d_B','TLFB90d_B','Sess_Type_Dual','Sess_Type_First Single','Sess_Type_Second single','Sess_Type_Steady','BIOSEX_0','Eth_Id_C_African/Af-Amer','Eth_Id_C_Asian','Eth_Id_C_Caucasian','Eth_Id_C_Hispanic','Eth_Id_C_Mixed']]
y=y2

random_forest(array,y)    

In [ ]:
no_categ_cols=['HT', 'WT', 'BF', 'VF', 'RM', 'WAIST', 'HIPS', 'AGE','CAL_LUN', 'ATE_C', 'WATER_C', 'TLFB90d_F','TLFB90d_B', 'Peak_TAC', 'TAC_Dur', 'TAC_AUC','TAC_0toPeak', 'TAC1', 'TAC2', 'TAC3', 'TAC4','BIOSEX_0'] 
array=X[no_categ_cols]
y=y_biv

random_forest(array,y)

HYPERPARAMETER TUNING

Random Search

In [ ]:
from pprint import pprint

# Base model to tune
rf = RandomForestRegressor(random_state=1)
print('Parameters currently in use:\n')
pprint(rf.get_params())

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=1, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

print('Parameters of best model:\n')
print(rf_random.best_params_)

def evaluate(model, X_val, y_val):
    predictions = model.predict(X_val)
    errors = abs(predictions - y_val)
    mape = 100 * np.mean(errors / y_val)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Absolute Percent Error = {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('Mean absolute error =', mean_absolute_error(y_val, predictions))
    return accuracy


base_model = RandomForestRegressor(n_estimators = 100, random_state = 1)
base_model.fit(X_train, y_train)
best_random = rf_random.best_estimator_

print('Base model:')
base_accuracy = evaluate(base_model, X_test, y_test)
print('Random search best model:')
random_accuracy = evaluate(best_random, X_test, y_test)
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2,3],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [800, 1000, 1200, 1400, 1600]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
print('Parameters of best model:\n')
print(grid_search.best_params_)
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

# KNN REGRESSION

In [73]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from numpy import absolute

TRAIN TEST SPLIT AND EVALUATION

In [ ]:
y=y1
X_tt=X[['TAC_0toPeak','TAC1', 'TAC4','Sess_Type_Dual']]
model = KNeighborsRegressor(n_neighbors=6, weights='distance')

test_tr_split()

CROSS VALIDATION FOR DIFFERENT K VALUES

In [152]:
def plot_complexity_curve(k_list, knn_model, weight, X_cv, y):
    
    cv_scores = []
    
    for k in k_list:
        knn = knn_model(k, weights=weight)
        cv = KFold(n_splits=5, shuffle=True, random_state=1)
        scores = cross_val_score(knn, X_cv, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
        cv_scores.append(mean(absolute(scores)))
    print(cv_scores)
    print('CV smallest MSE: ', min(cv_scores))
    fig, ax = plt.subplots(figsize=(7,7))
    ax.plot(k_list, cv_scores, label='Cross Validation MSE', color='black')
    ax.set(title='k-NN with Different Values for $k$', xlabel='Number of Neighbors', ylabel='CV MSE')
    ax.legend()

In [ ]:
neighbors = np.arange(1, 15)
weight='distance'
X_cv=X[['Peak_TAC','TAC_0toPeak', 'TAC1', 'TAC2', 'TAC4', 'Sess_Type_Dual']]
y=y1

plot_complexity_curve(neighbors, KNeighborsRegressor, weight, X_cv, y)

FORWARD/BACKWARD FEATURE SELECTION (FIX K AND WEIGHTS)

In [ ]:
feat_num=3
fb=False
X_fb=X[x_columns]
y=y2
model = KNeighborsRegressor(n_neighbors=9, weights='uniform')

feature_selection(X_fb,y,model,feat_num,fb)

GRID SEARCH FOR HYPERPARAMETER TUNING

In [56]:
from sklearn.model_selection import GridSearchCV

In [ ]:
print('Parameters currently in use:\n')
print(KNeighborsRegressor().get_params())

In [ ]:
y=y1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
parameters = {"n_neighbors": range(1, 15), "weights": ["uniform", "distance"]}

gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

print('Parameters of best model:\n', gridsearch.best_params_)
test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
print('Test MSE: ', test_mse)

PCA

In [28]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2','principal component 3', 'principal component 4'])
pca.explained_variance_ratio_

LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=5)
ldaReduction = lda.fit(X)
ldaDf = pd.DataFrame(data = ldaReduction, columns = ['col1', 'col2','col3','col4','col5'])
X_pca = pd.concat([ldaDf, df[['target']]], axis = 1)
transformed = lda.transform(data)

# BIVARIATE MODELS

# REGRESSOR CHAINS

In [17]:
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LinearRegression

PREDICTIONS ON SPECIFIC SESSIONS

In [ ]:
order=[0,1]
x_cols= ['CAL_LUN', 'Peak_TAC', 'TAC_0toPeak', 'TAC1', 'TAC3', 'TAC4', 'Sess_Type_Dual', 'BIOSEX_0']
X_train=X[x_cols]
X_test=X_test[x_cols]
lm=LinearRegression()
chain = RegressorChain(base_estimator=lm, order=order).fit(X_train, y_biv)
chain.predict(X_test)

FORWARD/BACKWARD FEATURE SELECTION

In [ ]:
feat_num=8
fb=True
x_columns=[ 'HT', 'WT', 'BF', 'VF', 'RM', 'WAIST', 'HIPS', 'AGE', 'mLALC','CAL_LUN', 'ATE_C', 'WATER_C', 'TLFB28d_F', 'TLFB90d_F', 'TLFB28d_B','TLFB90d_B', 'Peak_TAC', 'TAC_Dur', 'TAC_AUC', 'TAC_0toPeak', 'TAC1','TAC2', 'TAC3', 'TAC4', 'Sess_Type_Dual', 'Sess_Type_First Single','Sess_Type_Second single', 'Sess_Type_Steady', 'BIOSEX_0','Eth_Id_C_African/Af-Amer', 'Eth_Id_C_Asian', 'Eth_Id_C_Caucasian','Eth_Id_C_Hispanic', 'Eth_Id_C_Mixed']
X_fb=X[x_columns]
y=y_biv
lm=LinearRegression()
ordr=[1,0]
model=RegressorChain(base_estimator=lm, order=ordr)

feature_selection(X_fb,y,model,feat_num,fb)

K-FOLD CROSS VALIDATION

In [ ]:
x_cols=['TLFB28d_F', 'Peak_TAC', 'TAC_0toPeak', 'TAC1', 'TAC3', 'TAC4', 'Sess_Type_Second single', 'BIOSEX_0']
X_cv=X[x_cols]
y=y_biv
order=[0,1]
lm=LinearRegression()
model = RegressorChain(base_estimator=lm, order=order)

cross_val(X_cv,y,model)

# NEURAL NETS

In [ ]:
!pip install tensorflow
!pip install keras
!pip install scikeras
!pip install delayed

In [57]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor

K-FOLD CROSS VALIDATION

In [84]:
nnmodel = Sequential()
un=12
input_d=len(X.columns)
ep=30
b_size=20

nnmodel.add(Dense(units=un, input_dim=input_d, kernel_initializer='normal', activation='relu', kernel_regularizer=keras.regularizers.l2()))
nnmodel.add(Dense(units=un, kernel_initializer='normal', activation='relu', kernel_regularizer=keras.regularizers.l2()))
nnmodel.add(Dense(1, kernel_initializer='normal'))
nnmodel.compile(loss='mean_squared_error', optimizer='adam')
 
neural_network = KerasRegressor(build_fn=nnmodel, epochs=ep, batch_size=b_size, verbose=1)

In [ ]:
x_columns=['HT', 'WT', 'BF', 'VF', 'RM', 'WAIST', 'HIPS', 'AGE', 'mLALC','CAL_LUN', 'ATE_C', 'WATER_C', 'TLFB28d_F', 'TLFB90d_F', 'TLFB28d_B','TLFB90d_B', 'Peak_TAC', 'TAC_Dur', 'TAC_AUC', 'TAC_0toPeak', 'TAC1','TAC2', 'TAC3', 'TAC4', 'Sess_Type_Dual', 'Sess_Type_First Single','Sess_Type_Second single', 'Sess_Type_Steady', 'BIOSEX_0','Eth_Id_C_African/Af-Amer', 'Eth_Id_C_Asian', 'Eth_Id_C_Caucasian','Eth_Id_C_Hispanic', 'Eth_Id_C_Mixed']
X_cv=X[x_columns]
y=y1
model=neural_network

cross_val(X_cv,y,model)

# BOOTSTRAP

In [184]:
from scipy.io import savemat

Be careful! Need to change the the index for X_boots_1 and X_boots 2 for different test episode

In [234]:
def boots(q1_covs,q2_covs,model,new_ind_num,new_ind):
    
    q1_preds=[]
    q2_preds=[]
    q1_preds_cut=[]
    q2_preds_cut=[]

    X_boots_1=X[q1_covs]
    X_boots_2=X[q2_covs]
    X_new_1=X_boots_1.iloc[new_ind_num]
    X_new_2=X_boots_2.iloc[new_ind_num]
    X_new_1=X_new_1.to_numpy().reshape(1,-1)
    X_new_2=X_new_2.to_numpy().reshape(1,-1)
    X_boots_1=X_boots_1.drop(new_ind)
    X_boots_2=X_boots_2.drop(new_ind)
    q1_boots=y1[1::]
    q2_boots=y2[1::]

    rows=len(X_boots_1.index)
    sample=np.arange(start=0, stop=rows, step=1)
    
    #get 1000 q predictions using bootstrap 
    for iter in range(1000):
        x = np.random.choice(sample, size=rows, replace=True)
        X_train_1= X_boots_1.iloc[x]
        X_train_2= X_boots_2.iloc[x]
        q1_train= q1_boots[x]
        q2_train= q2_boots[x]
        lm_1 = model.fit(X_train_1, q1_train)
        q1_preds.append(lm_1.predict(X_new_1)[0])
        lm_2 = model.fit(X_train_2, q2_train)
        q2_preds.append(lm_2.predict(X_new_2)[0])  
    
    #get 1000 q predictions using bootstrap to calculate confidence bounds
    for iter in range(1000):
        x = np.random.choice(sample, size=rows, replace=True)
        X_train_1= X_boots_1.iloc[x]
        X_train_2= X_boots_2.iloc[x]
        q1_train= q1_boots[x]
        q2_train= q2_boots[x]
        lm_1 = model.fit(X_train_1, q1_train)
        q1_preds_cut.append(lm_1.predict(X_new_1)[0])
        lm_2 = model.fit(X_train_2, q2_train)
        q2_preds_cut.append(lm_2.predict(X_new_2)[0])
        
    #get the q estimate when using the full training set
    lm_1_true=model.fit(X_boots_1,q1_boots)
    q1_true=lm_1_true.predict(X_new_1)
    print(q1_true)
    lm_2_true=model.fit(X_boots_2,q2_boots)
    q2_true=lm_2_true.predict(X_new_2)
    print(q2_true)
    
    savemat("q1_predictions.mat", {"q1_preds": q1_preds})
    savemat("q2_predictions.mat", {"q2_preds": q2_preds})
    savemat("q1_predictions_cut.mat", {"q1_preds_cut": q1_preds_cut})
    savemat("q2_predictions_cut.mat", {"q2_preds_cut": q2_preds_cut})
    savemat("q1_true.mat",{"q1_true": q1_true})
    savemat("q2_true.mat",{"q2_true": q2_true})    

In [ ]:
q1_covs=['WT', 'BF', 'VF', 'RM', 'TLFB90d_F', 'Peak_TAC', 'TAC_AUC', 'TAC1', 'TAC4', 'Sess_Type_Second single', 'Eth_Id_C_Hispanic']
q2_covs=['HT', 'WT', 'BF', 'mLALC', 'ATE_C', 'TLFB90d_B', 'TAC_AUC', 'TAC_0toPeak', 'TAC3', 'Sess_Type_First Single', 'BIOSEX_0', 'Eth_Id_C_African/Af-Amer', 'Eth_Id_C_Caucasian', 'Eth_Id_C_Hispanic']
model=LinearRegression()
new_ind='BT306_S4R'
new_ind_num=0

boots(q1_covs,q2_covs,model,new_ind_num,new_ind)

In [ ]:
from scipy.io import savemat

savemat("q1_predictions.mat", {"q1_preds": q1_preds})
savemat("q2_predictions.mat", {"q2_preds": q2_preds})
savemat("q1_predictions_cut.mat", {"q1_preds_cut": q1_preds_cut})
savemat("q2_predictions_cut.mat", {"q2_preds_cut": q2_preds_cut})
savemat("q1_true.mat",{"q1_true": q1_true})
savemat("q2_true.mat",{"q2_true": q2_true})

# CLUSTERING THE Qs USING KMEANS

In [ ]:
conda install -c conda-forge kneed

In [222]:
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler

SCALE THE Q VALUES

In [51]:
X_kmeans=covariates[['q1','q2']]
X_kmeans=X_kmeans.iloc[22::]
#X_kmeans=X_kmeans[(X_kmeans['q1']<2.2) & (X_kmeans['q2']<2.5)]
data_with_clusters = X_kmeans.copy()
scaler = MinMaxScaler()
X_kmeans['q1']= scaler.fit_transform(X_kmeans['q1'].values.reshape(-1, 1))
X_kmeans['q2']= scaler.fit_transform(X_kmeans['q2'].values.reshape(-1, 1))

APPLY KMEANS AND CALCULATE THE PERCENTAGE OF EACH CLUSTER

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=3, n_init=1000, max_iter=300, random_state=42)
kmeans.fit(X_kmeans)
identified_clusters = kmeans.fit_predict(X_kmeans)
count_0 = len(identified_clusters[identified_clusters==0])
count_1 = len(identified_clusters[identified_clusters==1])
count_2 = len(identified_clusters[identified_clusters==2])
pct_of_0 = count_0/(count_0+count_1+count_2)
print("percentage of 0 cluster is", pct_of_0*100)
pct_of_1 = count_1/(count_0+count_1+count_2)
print("percentage of 1 cluster", pct_of_1*100)
pct_of_2 = count_2/(count_0+count_1+count_2)
print("percentage of 2 cluster", pct_of_2*100)
data_with_clusters['Clusters'] = identified_clusters 
plt.scatter(data_with_clusters['q1'],data_with_clusters['q2'],c=data_with_clusters['Clusters'],cmap='rainbow')
plt.xlabel('q1')
plt.ylabel('q2')
print('The lowest SSE value:', kmeans.inertia_)
print('Final locations of the centroid:', kmeans.cluster_centers_)
print('The number of iterations required to converge:', kmeans.n_iter_)

SAVE THE INDEXES FOR EACH CLUSTER

In [76]:
from scipy.io import savemat

index_0=list(np.where(identified_clusters==0))[0]
index_1=list(np.where(identified_clusters==1))[0]
index_2=list(np.where(identified_clusters==2))[0]
matlab_index=covariates['entry'].iloc[22::]
index_0_mat=matlab_index[index_0].values
index_1_mat=matlab_index[index_1].values
index_2_mat=matlab_index[index_2].values
savemat("index_0.mat", {"ind_0": index_0_mat})
savemat("index_1.mat", {"ind_1": index_1_mat})
savemat("index_2.mat", {"ind_2": index_2_mat})

SAVE THE Q VALUES WITH THE CORRESPONDING MATLAB INDEX

In [70]:
q_values=covariates[['entry','q1','q2']].iloc[22::].values
savemat("q_values.mat", {"q_values": q_values})

SAVE THE INDEXES OF THE FULL DATASET 

In [77]:
train_index_full=covariates.entry.iloc[22::].values
savemat("train_ind_full.mat", {"tr_ind_full": train_index_full})

# BEST NUMBER OF CLUSTERS

ELBOW METHOD

In [ ]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
    }

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X_kmeans)
    sse.append(kmeans.inertia_)
    
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
    
kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
kl.elbow

SILHOUETTE COEFFICIENT

In [ ]:
# A list holds the silhouette coefficients for each k
silhouette_coefficients = []
# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X_kmeans)
    score = silhouette_score(X_kmeans, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

# BINOMIAL/MULTINOMIAL LOGISTIC REGRESSION 

In [46]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

OVERSAMPLING USING SMOTE, TRAIN THE MODEL AND EVALUATE ON TEST SESSIONS

In [ ]:
conda install -c conda-forge imbalanced-learn

In [223]:
def oversampling(db, drop_ind,data_with_clusters):

    X_new=db[db['entry'].isin(drop_ind)].drop('entry',1)
    y_train = data_with_clusters.Clusters[~data_with_clusters.index.isin(X_new.index)].values
    X_train = X[~X.index.isin(X_new.index)].drop('entry',1)
    os = SMOTE(random_state=1)
    columns = X_train.columns
    os_data_X,os_data_y=os.fit_sample(X_train, y_train)
    os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
    os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

    print("length of oversampled data is ",len(os_data_X))
    print("Number of 0 in oversampled data",len(os_data_y[os_data_y['y']==0]))
    print("Number of 1 in oversampled data",len(os_data_y[os_data_y['y']==1]))
    print("Number of 2 in oversampled data",len(os_data_y[os_data_y['y']==2]))
    print("Proportion of 0 data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
    print("Proportion of 1 in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))
    print("Proportion of 2 in oversampled data is ",len(os_data_y[os_data_y['y']==2])/len(os_data_X))
    
    return os_data_X, os_data_y

In [ ]:
db=X
drop_ind=[34, 50, 82]

oversampling(db, drop_ind,data_with_clusters)

In [ ]:
os_data_y=os_data_y.values.ravel()
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000, penalty='none')
model.fit(os_data_X, os_data_y)
y_pred = model.predict(X_new)
print('Predicted class:' , y_pred)

TRAIN/TEST SPLIT AND EVALUATION USING OVERSAMPLING 

K-FOLD CROSS VALIDATION MULTINOMIAL LOGISTIC REGRESSION

In [35]:
def multinomial_log_reg_cv(X,y):
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
multinomial_log_reg_cv(os_data_X,os_data_y)

K-FOLD CROSS VALIDATION BINOMIAL LOGISTIC REGRESSION

In [83]:
def binomial_log_reg_cv(X,y):
    model = LogisticRegression(solver='lbfgs', penalty='l2', C=1)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
binomial_log_reg_cv(os_data_X,os_data_y)

BINOMIAL LOGISTIC REGRESSION TRAIN/TEST SPLIT AND EVALUATION

In [56]:
def binomial_tt(X_tt,y_tt,X_te,y_te):
    model = LogisticRegression(solver='lbfgs', max_iter=1000, penalty='l2', C=1)
    model.fit(X_tt, y_tt)
    y_pred = model.predict(X_te)

    res = "\n".join("{} {}".format(x, y) for x, y in zip(y_te, y_pred))
    print(res)
    print('Accuracy Score:', metrics.accuracy_score(y_te, y_pred)) 
    class_report=classification_report(y_te, y_pred)
    print(class_report)
    confusion_matrix(y_te, y_pred)
    confmtrx = np.array(confusion_matrix(y_te, y_pred))
    pd.DataFrame(confmtrx, index=['0','1'], columns=['predicted_0', 'predicted_1'])

In [ ]:
X_tt=os_data_X
y_tt=os_data_y.values.ravel()
X_te=X_test
y_te=y_test.ravel()
binomial_tt(X_tt,y_tt,X_te,y_te)

MULTINOMIAL LOGISTIC REGRESIION TRAIN/TEST SPLIT AND EVALUATION

In [85]:
def multinomial_tt(X_tt,y_tt,X_te,y_te):
    #y=identified_clusters 
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1)
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, penalty='none')
    model.fit(X_tt, y_tt)
    y_pred = model.predict(X_te)

    res = "\n".join("{} {}".format(x, y) for x, y in zip(y_te, y_pred))
    print(res)
    print('Accuracy Score:', metrics.accuracy_score(y_te, y_pred)) 
    class_report=classification_report(y_te, y_pred)
    print(class_report)
    confusion_matrix(y_te, y_pred)
    confmtrx = np.array(confusion_matrix(y_te, y_pred))
    print(pd.DataFrame(confmtrx, index=['0','1','2'], columns=['predicted_0', 'predicted_1','predicted_2']))

In [ ]:
X_tt=os_data_X
y_tt=os_data_y.values.ravel()
X_te=X_test
y_te=y_test.ravel()
multinomial_tt(X_tt,y_tt,X_te,y_te)

MULTINOMIAL LOGISTIC REGRESSION TUNE FOR THE WEIGHTING OF PENALTY

In [ ]:
def get_models():
    models = dict()
    for p in [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
        key = '%.4f' % p
        if p == 0.0:
            models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none')
        else:
            models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=p)
    return models
 
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, os_data_X, os_data_y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

MULTINOMIAL LOGISTIC REGRESSION TUNE FOR THE WEIGHTING OF PENALTY

In [ ]:
def get_models():
    models = dict()
    for p in [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
        key = '%.4f' % p
        if p == 0:
            models[key] = LogisticRegression(solver='lbfgs', penalty='none')
        else:
            models[key] = LogisticRegression(solver='lbfgs', penalty='l2', C=p)
    return models
 
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, os_data_X, os_data_y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()          

# KNN CLASSIFICATION

In [23]:
from sklearn.neighbors import KNeighborsClassifier

TRAIN/TEST SPLIT AND EVALUATION

In [54]:
def KNN_tt(X_tt,y_tt,X_te,y_te,k,weight):
    model = KNeighborsClassifier(n_neighbors = k, weights=weight)
    model.fit(X_tt, y_tt)
    y_pred = model.predict(X_te)

    res = "\n".join("{} {}".format(x, y) for x, y in zip(y_te, y_pred))
    print(res)
    print('Accuracy Score:', metrics.accuracy_score(y_te, y_pred)) 
    class_report=classification_report(y_te, y_pred)
    print(class_report)
    confusion_matrix(y_te, y_pred)
    confmtrx = np.array(confusion_matrix(y_te, y_pred))
    print(pd.DataFrame(confmtrx, index=['0','1','2'], columns=['predicted_0', 'predicted_1','predicted_2']))

In [ ]:
X_tt=os_data_X
y_tt=os_data_y.values.ravel()
X_te=X_test
y_te=y_test.ravel()

KNN_tt(X_tt,y_tt,X_te,y_te,11,'distance')

K-FOLD CROSS VALIDATION

In [50]:
def KNN_classification_cv(X_cv,y_cv,k,weight):
    model = KNeighborsClassifier(n_neighbors = k, weights=weight)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X_cv, y_cv, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
KNN_classification_cv(os_data_X, os_data_y,11,'distance')

COMPLEXITY CURVE

In [46]:
def plot_complexity_curve_classification(k_list, knn_model, weight, X_cv, y_cv):
    
    cv_scores = []
    
    for k in k_list:
        knn = knn_model(k, weights=weight)
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
        scores = cross_val_score(knn, X_cv, y_cv, scoring='accuracy', cv=cv, n_jobs=-1)
        cv_scores.append(mean(scores))
    
    print('CV Largest Accuracy: ', max(cv_scores))
    fig, ax = plt.subplots(figsize=(7,7))
    ax.plot(k_list, cv_scores, label='Cross Validation Accuracy', color='black')
    ax.set(title='k-NN with Different Values for $k$', xlabel='Number of Neighbors', ylabel='CV Accuracy')
    ax.legend()

In [ ]:
neighbors = np.arange(1, 15)
weight='distance'

plot_complexity_curve_classification(neighbors, KNeighborsClassifier, weight, os_data_X, os_data_y)